In [ ]:
%matplotlib inline

In [ ]:
import json
import os
import time
from copy import copy
from functools import partial
from os.path import basename, join
from subprocess import call
from urllib.request import urlopen
import numpy as np
import pyproj
import requests

import geopandas as gpd
import pandas as pd
import us
from shapely.geometry import LineString, Point, shape
from shapely.ops import transform

from covidcaremap.data import external_data_path, processed_data_path

DH facilities are geocoded using the [Google Maps](https://developers.google.com/maps/documentation/geocoding/start) and [Mapbox](https://docs.mapbox.com/api/search/) geocoding APIs. Google Maps is the primary source while Mapbox is used as a fallback option. We validate the geocoding results by checking their spatial overlap with administrative regions (states, counties, zip codes) that they are expected to fall into. We use three different level because of underlying inaccuracies in the DH data.

Geocoding steps:
- Construct a query based on address info in the DH record
- Find the geometries that the output point is expected to fall into based on the DH record(state, county, zip code)
- Geocode using the Google Maps API
- If the point falls into the expected County or Zip Code, proceed, otherwise check it against its expected state geometry.
- Track the source of confirmation for each facility so that we can manually inspect the results that were confirmed at the higher geographic level (i.e. state)

### Input Datasets

*states geodataframe*

In [ ]:
states = gpd.read_file('https://gist.githubusercontent.com/simonkassel/d091fc86253b65c68bb644443c74f366/raw/001b6ad8232e2ecfdc5fbd46a5ef8f2a9642e94d/us_states.geojson')
states.rename(columns={'NAME': 'ST_NAME', 'abbr': 'ST_ABBR', 'STATEFP':'STATE_FP'}, inplace=True)
states['ST_NAME'] = states['ST_NAME'].apply(lambda x: x.upper())

*counties geodataframe*

In [ ]:
def gdf_from_geojson(geojson, crs='epsg:4326'):
    """Convert a GeoJSON dict to GeoDataFrame"""
    def _f(f):
        f['properties']['geometry'] = shape(f['geometry'])
        return f['properties']
    
    return gpd.GeoDataFrame([_f(f) for f in geojson['features']], crs=crs)

fips_to_abbr = {st.fips: st.abbr for st in us.states.STATES_AND_TERRITORIES if st.fips}

In [ ]:
with urlopen("https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_20m.json") as url:
    data = json.loads(url.read().decode("ISO-8859-1"))
counties = gdf_from_geojson(data)
counties.rename(columns={'STATE':'STATE_FP', 'COUNTY': 'COUNTY_FP', 'NAME': 'COUNTY_NAME'}, inplace=True)
counties['COUNTY_NAME'] = counties['COUNTY_NAME'].apply(lambda x: x.upper())
counties['ST_ABBR'] = counties['STATE_FP'].apply(lambda x: fips_to_abbr[x])

*zip code convex hull geodataframe*

The zip code geojson file for the whole US was prohibitively large so we reduced the size by simplifying the polygons into their [convex hulls](https://en.wikipedia.org/wiki/Convex_hull). This dramatically reduced the file size while keeping enough spatial information for the simple task of validating basic location.

In [ ]:
zip_convex_hulls = gpd.read_file(external_data_path('us_zip_codes-convex_hulls.geojson'))

*DH dataframe*

In [ ]:
dh = gpd.read_file(external_data_path('dh_facility_data.geojson'), encoding='utf-8')
dh = dh[~dh['geometry'].isna()]
dh.reset_index(inplace=True)
dh_orig = dh.copy()
# use 5 digit zip codes to match the us zip code geojson
dh['HQ_ZIP_COD'] = dh['HQ_ZIP_COD'].apply(lambda x: x.split('-')[0])
dh.rename(columns = {'HQ_CITY': 'CITY_NAME', 
                     'HQ_STATE': 'ST_ABBR', 
                     'COUNTY_NAM': 'COUNTY_NAME', 
                     'HQ_ZIP_COD': 'ZIP_CODE',
                     'HOSPITAL_N': 'HOSP10_Name',
                     'HQ_ADDRESS': 'Street_Addr'}, inplace=True)

### geocoding

In [ ]:
# Either replace the empty strings in the subsequent two lines with the appropriate API keys or set them as 
# environment variables
google_key = ''
mapbox_key = ''

google_key = os.getenv('GOOGLE_API_KEY', google_key)
mapbox_key = os.getenv('MAPBOX_API_KEY', mapbox_key)

In [ ]:
def google_geocode_str(name, addr, city, state, county, zip_code, key, condense=False, address_only=False):
    """
    Generate a Google Maps query url from DH fields and api key
    
    `condense` is an optional boolean parameter that enables construction of the URL with 
    all of the components combined into the address field rather than as separate components
    this is an option if the latter fails
    """
    
    country = 'US'
    if state in ('PR', 'GU', 'MP'):
        country = state

    base = 'https://maps.googleapis.com/maps/api/geocode/json?'
    if condense:
        if address_only:
            address_str = f'address={name}, {addr}'
        else:
            address_str = f'address={name}, {addr}, {city}, {state}, {county} county, {zip_code}'
        components_str = '&components=country:{}'.format(country)
    else:
        address_str = 'address={}, {}'.format(name, addr)
        if address_only:
            components_str = f'&components=country:{country}'
        else:
            components_str = f'&components=country:{country}|locality:{city}|administrative_area:{state}|administrative_area:{county} county|postal_code:{zip_code}'
    
    key_str = '&key={}'.format(key)
    
    return base + address_str + components_str + key_str

In [ ]:
def mapbox_geocode_str(name, addr, city, state, county, zip_code, key):
    """
    Generate a Mapbox query url from DH fields and api key
    """
    country = 'US'
    if state in ('PR', 'GU', 'MP'):
        country = state
        
    base = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
    address_str = f'{name},{addr},{city},{state},{county} county,{zip_code}.json?'
    components_str = 'country={}&limit=5'.format(country)
    key_str = '&access_token={}'.format(key)
    
    query_str = base + address_str + components_str + key_str
    # mapbox urls are limited to 256 characters or fewer. Strategically remove component
    # of the address if necessary to get under the threshold
    if len(query_str) > 256:
        query_str = query_str.replace('{},'.format(name), '')
        if len(query_str) > 256:
            query_str = query_str.replace('{} county,'.format(county), '')
    
    return query_str

In [ ]:
# Change this test flag to `True` in order to run a smaller subset of all DH facilities
# to make sure everythin is worlking correctly
test = False
if test:
    dh = dh.copy()
    dh = dh.head(100)

iterate over all facilities storing the geocode results and background information

In [ ]:
dhs = dh.copy()

In [ ]:
start = 0
finish = 6600
if finish > np.max(dh.index):
    finish = np.max(dh.index) + 1

dhs = dh.iloc[start:finish].copy()

geometry = []
source = []
confs = []

for k, r in dhs.iterrows():
    county_geom = None
    zip_geom = None
    state_geom = None
    
    if isinstance(r['COUNTY_NAME'], str):
        county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == r['COUNTY_NAME'])]
        if len(county) == 1:
            county_geom = county['geometry'].values[0]
        
        elif len(county) < 1:
            # some counties are encoded slightly differently in dh dataset
            rcn = r['COUNTY_NAME'].replace(' COUNTY', '')
            county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == rcn)]
            if len(county) == 1:
                county_geom = county['geometry'].values[0]
        
        elif len(county) > 1:
            county = county[county['LSAD'] == 'County']
            if len(county) == 1:
                county_geom = county['geometry'].values[0]
    
    # using zip code is necessary because many facilities do not fall within counties or have
    # incorrect county labels in the dh dataset
    if isinstance(r['ZIP_CODE'], str):
        zips = zip_convex_hulls[zip_convex_hulls['ZIP_CODE'] == r['ZIP_CODE']]
        if len(zips) == 1:
            zip_geom = zips['geometry'].values[0]

    state_geom = states[states['ST_ABBR'] == r['ST_ABBR']]['geometry'].values[0]
    
    
    # workaround for one exception in PR
    if r['Street_Addr'] == 'ROAD 172 EXIT TO CIDRA':
        r['Street_Addr'] = ''
        
    args = [r['HOSP10_Name'], r['Street_Addr'], r['CITY_NAME'], r['ST_ABBR'], r['COUNTY_NAME'], r['ZIP_CODE']]
    args = [str(a).replace('#', '').replace('/', ' ').replace(';', '').replace('?', '') for a in args]
    
    def _p(search_str):
        """
        Convert http request str into a validated shapely point
        """
        try:
            response = requests.get(search_str)
        except ConnectionError:
            return 'disconnection'
        
        if response.status_code == 422:
            print('Mapbox query string too long, relying on google maps')
        
        if response.status_code == 500:
            print(search_str)
            for i in range(10):
                time.sleep(120)
                response = requests.get(search_str)
                print(i)
                if response.status_code != 500:
                    break
        
        if response.status_code != 200:
            print(response.json())
            print(search_str)
            raise Exception('Status code exeption: {}'.format(response.status_code))
        
        
        response = response.json()
        if 'results' in response:
            for result in response['results']:
                y, x = result['geometry']['location'].values()
                hosp_point = Point(x, y)
                if county_geom:
                    if hosp_point.within(county_geom):
                        return (hosp_point, 'county')
                if zip_geom:
                    if hosp_point.within(zip_geom):
                        return (hosp_point, 'zip code')
                if state_geom:
                    if hosp_point.within(state_geom):
                        return (hosp_point, 'state')
        else:
            for feature in response['features']:
                hosp_point = Point(feature['center'])
                if county_geom:
                    if hosp_point.within(county_geom):
                        return (hosp_point, 'county')
                if zip_geom:
                    if hosp_point.within(zip_geom):
                        return (hosp_point, 'zip code')
                if state_geom:
                    if hosp_point.within(state_geom):
                        return (hosp_point, 'state')
        
        return (None, None)
    
    # Google
    # Try google first
    o = _p(google_geocode_str(*args + [google_key]))
    if o == 'disconnect':
        break
    else:
        google_point, confirmation = o
        if not google_point:
            # and try both different types of google request construction
            o = _p(google_geocode_str(*args + [google_key]))
            if o == 'disconnect':
                break
            else:
                google_point, confirmation = _p(google_geocode_str(*args + [google_key, True]))
    
    # Mapbox
    # only try mapbox if google didn't return a valid point
    mapbox_point = None
    if not google_point:
        o = _p(mapbox_geocode_str(*args + [mapbox_key]))
        if o == 'disconnect':
            break
        else:
            mapbox_point, confirmation = o
        if not mapbox_point:
            o = google_point, confirmation = _p(google_geocode_str(*args + [google_key, True, True]))
            if o == 'disconnect':
                break
            else:
                google_point, confirmation = o
        
    if google_point:
        source.append('google')
        geometry.append(google_point)
    elif mapbox_point:
        source.append('mapbox')
        geometry.append(mapbox_point)
    else:
        print('No coordinates found for row:')
        print(r)
        print()
        source.append(None)
        geometry.append(None)

    confs.append(confirmation)
    
    if (k + 1) % 100 == 0:
        print('Geocoded [{}] of {} hospitals.'.format(k + 1, len(dh)))

if o == 'disconnect':
    print('Disconnected at row {}'.format(k))
    finish = k
    dhs = dh.iloc[start:finish].copy()

In [ ]:
# add geocoding info to dh dataset and convert into GeoDataFrame
dhs['source'] = source
dhs['geometry'] = geometry
dhs['confirmation_source'] = confs

In [ ]:
dhs_orig = dh_orig.iloc[start:finish].copy()

In [ ]:
for g in (dhs, dhs_orig):
    g.to_crs('EPSG:3857', inplace=True)

In [ ]:
dists = [x.distance(y) for x, y in list(zip(dhs['geometry'], dhs_orig['geometry']))]

In [ ]:
dhs['distance'] = dists
dhs_orig['distance'] = dists

In [ ]:
for g in (dhs, dhs_orig):
    g.to_crs('EPSG:4326', inplace=True)

In [ ]:
dhs.to_file(processed_data_path('dh_geocoded_v1_0326202.geojson'), driver='GeoJSON')
dhs_orig.to_file(processed_data_path('dh_orig_for_mapping_v1_0326202.geojson'), driver='GeoJSON')

Refactor geocoding:

In [ ]:
class GeoCoder:
    def __init__(self, 
                 google_api_key,
                 mapbox_api_key):
        self.google_api_key = google_api_key
        self.mapbox_api_key = mapbox_api_key

class DataSetGeocoder(GeoCoder):
    def __init__(self,
                 google_api_key,
                 mapbox_api_key,
                 dataset,
                 dataset_key=None, 
                 validation_dataset=None):
        
        super(GeoCoder).__init__(google_api_key, mapbox_api_key)
        self.dataset = dataset
        self.validation_dataset = validation_dataset
        self.dataset_key = dataset_key
        
        self.missing = []
        self.complete = []
    
    def geocode(self):
        for k, r in self.dataset.iterrows():
            name = self.dataset_key.get('name')
            addr = self.dataset_key.get('address')
            

class FacilityGeocoder(GeoCoder, DataSetGeocoder):
    def __init__(self,
                 dataset_geocoder,
                 name,
                 address,
                 city=None,
                 state=None,
                 county=None,
                 zip_code=None):
        self.ds = dataset_geocoder
        self.name = name
        self.address = address
        self.city = city
        self.state = state
        self.county = county
        self.zip_code = zip_code
    
    
    def google_geocode_str(self, condense=False):
        """
        Generate a Google Maps query url from DH fields and api key

        `condense` is an optional boolean parameter that enables construction of the URL with 
        all of the components combined into the address field rather than as separate components
        this is an option if the latter fails
        """

        country = 'US'
        if state in ('PR', 'GU', 'MP'):
            country = state

        base = 'https://maps.googleapis.com/maps/api/geocode/json?'
        address_str = f'address={self.name}, {self.addr}'
        if condense:
            for a in [self.city, self.state, self.county, self.zip_code]:
                if a:
                    if a == self.county:
                        a = '{} county'.format(self.county)
                    address_str += ', {}'.format(a)
            components_str = '&components=country:{}'.format(country)
        else:
            components_str = f'&components=country:{country}|locality:{city}|administrative_area:{state}|administrative_area:{county} county|postal_code:{zip_code}'

        key_str = '&key={}'.format(self.ds.gooogle_api_key)

        return base + address_str + components_str + key_str
    
    
    def mapbox_geocode_str(self):
        """
        Generate a Mapbox query url from DH fields and api key
        """
        country = 'US'
        if state in ('PR', 'GU', 'MP'):
            country = state

        base = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
        address_str = f'{name},{addr}.json?'
        for a in [self.city, self.state, self.county, self.zip_code]:
            if a:
                if a == self.county:
                    a = '{} county'.format(self.county)
            address_str.replace('.json?', ', {}.json?'.format(a))
            
        components_str = 'country={}&limit=5'.format(country)
        key_str = '&access_token={}'.format(self.ds.mapbox_api_key)

        query_str = base + address_str + components_str + key_str
        # mapbox urls are limited to 256 characters or fewer. Strategically remove component
        # of the address if necessary to get under the threshold
        if len(query_str) > 256:
            query_str = query_str.replace('{},'.format(name), '')
            if len(query_str) > 256:
                query_str = query_str.replace('{} county,'.format(county), '')

        return query_str